In [3]:
from sklearn import linear_model
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import time
import lightgbm as lgb
%matplotlib inline

# 폰트설정 (맥)
from matplotlib import font_manager
plt.rcParams["font.family"] = 'NanumGothicCoding'
from matplotlib import rc
rc('font', family="Arial Unicode MS")

# 폰트 설정 (윈도우)
from matplotlib import rc

rc('font', family='Malgun Gothic')

In [17]:
best = pd.DataFrame([[40, 500, 3767.2188255416995], [40, 470, 3773.734609249986],[40, 440, 3779.8886402420894],[25, 100, 3975.145895653563], [25, 120, 3946.5334015130366],[10, 100, 4103.34157617724], [10, 120, 4064.319256966652], [10, 140, 4041.888347803674], [10, 160, 4023.3257594292195], [10, 180, 4008.309806856634],[15, 100, 3981.9591316680703], [15, 120, 3951.086582162809], [15, 140, 3927.07648125976],[25, 300, 3838.1515559441086], [25, 280, 3850.427535864345],[15, 300, 3836.975855985116],
 [15, 270, 3842.699696838895],
 [15, 240, 3856.147983271992],
 [15, 210, 3870.1379072696036],
[70, 3000, 3663.809835262183], [60, 1000, 3740.97968704464],
 [60, 800, 3738.4109722560047],
 [60, 600, 3756.1232253862063],[70, 2500, 3670.7078738845394],[35, 100, 3961.4431866306863], [80, 5000, 3660.2213374375324], [80, 4500, 3665.869183414939]]).rename(columns={0:'max_depth', 1:'num_leaves', 2: 'RMSE'})

In [18]:
best.sort_values('RMSE')

,max_depth,num_leaves,RMSE
25,80,5000,3660.221337
19,70,3000,3663.809835
26,80,4500,3665.869183
23,70,2500,3670.707874
21,60,800,3738.410972
20,60,1000,3740.979687
22,60,600,3756.123225
0,40,500,3767.218826
1,40,470,3773.734609
2,40,440,3779.888640


In [19]:
train = pd.read_csv('datas/train.csv')


train['year'] =  round(train['transaction_year_month'] / 100)
train['month'] = train['transaction_year_month'] % 100


train = train[['transaction_id', 'apartment_id', 'city',  'dong', 'jibun', 'apt',
               'addr_kr', 'exclusive_use_area', 'year_of_completion',
               'transaction_year_month','year', 'month', 'transaction_date', 'floor',
               'transaction_real_price']]

FileNotFoundError: ignored

In [4]:
train_x = pd.read_csv('/train_xlgbm.csv')
train_y = pd.read_csv('/train_ylgbm.csv')

In [6]:
train_x.head()

,Unnamed: 0,dong2,apartment_id2,exclusive_use_area,year_of_completion,year,month,floor
0,0,신교동,7622,84.82,2002,2008.0,1,2
1,1,필운동,5399,99.17,1973,2008.0,1,6
2,2,필운동,3578,84.74,2007,2008.0,1,6
3,3,내수동,10957,146.39,2003,2008.0,1,15
4,4,내수동,10639,194.43,2004,2008.0,1,3


In [8]:
train_x["dong2"] = train_x["dong2"].astype('category')
train_x["apartment_id2"] = train_x["apartment_id2"].astype('category')
train_y['ppm2'] = train_y['transaction_real_price'] / train_x["exclusive_use_area"]

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [9]:
train_x = train_x[["dong2","apartment_id2","exclusive_use_area","year_of_completion","year","month","floor"]]
train_y = train_y[["ppm2"]]

In [12]:
train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size=0.2, random_state=13)

In [13]:
train_ds = lgb.Dataset(train_x, label = train_y) 
test_ds = lgb.Dataset(test_x, label = test_y) 

In [14]:
params = {'learning_rate': 0.01, 
          'max_depth': 80, 
          'boosting': 'gbdt', 
          'objective': 'regression', 
          'metric': 'mse', 
          'is_training_metric': True, 
          'num_leaves': 5000, 
          'feature_fraction': 0.9, 
          'bagging_fraction': 0.7, 
          'bagging_freq': 5, 
          'seed':2018}

In [15]:
model = lgb.train(params, train_ds, 10000, test_ds, verbose_eval=100, early_stopping_rounds=100)

/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 14430.2
[200]	valid_0's l2: 3620.39
[300]	valid_0's l2: 1840.1
[400]	valid_0's l2: 1454.61
[500]	valid_0's l2: 1327.48
[600]	valid_0's l2: 1268.97
[700]	valid_0's l2: 1233.85
[800]	valid_0's l2: 1212.76
[900]	valid_0's l2: 1199.3
[1000]	valid_0's l2: 1190.67
[1100]	valid_0's l2: 1184.75
[1200]	valid_0's l2: 1180.34
[1300]	valid_0's l2: 1177.24
[1400]	valid_0's l2: 1175.2
[1500]	valid_0's l2: 1173.71
[1600]	valid_0's l2: 1172.42
[1700]	valid_0's l2: 1171.86
[1800]	valid_0's l2: 1171.26
Early stopping, best iteration is:
[1774]	valid_0's l2: 1170.9


In [16]:
predict_test = model.predict(test_x)

In [17]:
mse = mean_squared_error(test_y, predict_test)
r2 = r2_score(test_y, predict_test)
rmse = mean_squared_error(test_y, predict_test)**0.5

In [18]:
print('Mean squared error: ', mse)
print('R2 score: ', r2)
print('RMSE score:', rmse)

Mean squared error:  1170.9034954098092
R2 score:  0.9867032659809915
RMSE score: 34.21846716920279
